In [1]:
import requests 

from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from time import sleep
import pandas as pd
import numpy as np
import datetime
import os
import pickle

# Inicializar parametros de entrada

In [76]:
# PONER USER-AGENT PROPIO:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"

# Options:
opts = Options()
opts.add_argument("--disable-extensions")
opts.add_argument(f"User-Agent={user_agent}")

# Cargar el diccionario de bullets
with open('dic_bullets.pickle', 'rb') as bullets:
    bullets_dic = pickle.load(bullets)

# URL talenthacker:
url_talenthacker = "https://talenthackers.net/spots/"

# Función que me devuelve la URL de cada oferta de empleo

In [11]:
def url_scraper(url):
    
    # Inicio navegador
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()    
    browser.get(url)
    sleep(2)
    
    # Acepto cookies:
    browser.find_element(By.CLASS_NAME, "q-banner__actions").find_element(By.CLASS_NAME, "buttons-wrapper").find_element(By.CLASS_NAME, "q-btn--standard").click()

    # Hago scroll hasta poder ver todas las ofertas disponibles:
    last_height = browser.execute_script("return document.documentElement.scrollHeight")
    while True:
        # Scroll:
        browser.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")
        sleep(3)
        # Calcula la altura del nuevo scroll y la compara con la del último:
        new_height = browser.execute_script("return document.documentElement.scrollHeight")

        if new_height == last_height:
            print("Ya no hay más página.")
            break
        last_height = new_height

    soup = BeautifulSoup(browser.page_source, "html.parser")

    talent_hacker = "https://talenthackers.net"
    ofertas = soup.find("div", class_= "jobs-list-wrapper").find_all("div", class_= "col-12")

    url_ofertas = []
    for oferta in ofertas:
        url_oferta = talent_hacker + oferta.find("a")["href"]
        url_ofertas.append(url_oferta)
        
    return url_ofertas

# Saco la info de cada empleo:

In [111]:
def iniciar_scraper(url_empleos, bullet_point= bullets_dic):
    # Inicio browser:
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    
    # Valores fijos:
    portal = "talenthacker"
    fecha_scrapeo = datetime.datetime.now().date()
    
    
    datos_ofertas_empleo = []
    contador = 0
    for empleo in url_empleos:
        
        browser.get(empleo)
        sleep(2)
        
        try:
            # Acepto cookies
            browser.find_element(By.CLASS_NAME, "q-banner__actions").find_element(By.CLASS_NAME, "buttons-wrapper").find_element(By.CLASS_NAME, "q-btn--standard").click()
        except:
            pass
        
        soup_oferta = BeautifulSoup(browser.page_source, "html.parser")
        
        #########################  INFORMACION  ##########################################################################
        
        try:
            titulo = soup_oferta.find("span", class_= "spot-title").text
        except:
            titulo = np.nan
            
        try:
            herramientas = [herramienta.text for herramienta in soup_oferta.find("div", class_= "spot-skills").find_all("a")]
        except:
            herramientas = np.nan
            
        try:
            descripcion = soup_oferta.find_all("div", class_= "block full-width text-body2")[0].text
        except:
            descripcion = np.nan
            
        try:
            funciones = soup_oferta.find_all("div", class_= "block full-width text-body2")[1].text
        except:
            funciones = np.nan
            
            
        imagenes = soup_oferta.find_all("div", class_= "th-body-md flex no-wrap q-ma-sm")
        for imagen in imagenes:
            img = imagen.find("img")["src"]
            
            if img == bullet_point["ubicacion"]:
                try:
                    ubicacion = imagen.text
                except:
                    ubicacion = np.nan
                    
            elif img == bullet_point["presencialidad"]:
                try:
                    presencialidad = imagen.text.strip()
                except:
                    presencialidad = np.nan
                    
            elif img == bullet_point["jornada_tipo"]:
                try:
                    tipo_contrato = imagen.text.split("·")[0].strip()
                    jornada = imagen.text.split("·")[1].strip()
                except:
                    tipo_contrato = np.nan 
                    jornada = np.nan
            
            if img == bullet_point["experiencia"]:
                try:
                    experiencia = imagen.text.strip()
                except:
                    experiencia = np.nan

            
            elif img == bullet_point["salario"]:
                try:
                    salario = imagen.text.strip()
                except:
                    salario = np.nan

        
        ###################################################################################################################
        datos_ofertas_empleo.append([titulo, herramientas, descripcion, ubicacion, presencialidad, funciones, jornada, experiencia, tipo_contrato, salario, fecha_scrapeo, empleo, portal])
        contador += 1

        if contador % 100 == 0:
            sleep(20)
            
            
    df = pd.DataFrame(datos_ofertas_empleo, columns= ["titulo", "herramientas", "descripcion", "ubicacion", "presencialidad", "funciones", "jornada", "experiencia", "tipo_contrato", "salario", "fecha_scrapeo", "url", "portal"])
    
    return df 

# Extraigo la url de cada empleo en la plataforma Infoempleo:

In [12]:
url_ofertas = url_scraper(url_talenthacker)

Ya no hay más página.


# Extraigo la información de cada oferta de empleo:

In [112]:
df = iniciar_scraper(url_ofertas)

# Lo guardo en la ruta de datos:
fecha = datetime.datetime.now().strftime("%d-%m-%y")
ruta_datos = "C:\\Users\\regue\\Desktop\\Data Science Projects\\PROJECTS\\IT_Job_Spain_Project\\Datos\\datos_sin_procesar\\"
nombre_archivo = f"datos_talenthacker_{fecha}.csv"
ruta_completa = os.path.join(ruta_datos, nombre_archivo)

df.to_csv(ruta_completa, index=False, sep=",")

In [113]:
df

,titulo,herramientas,descripcion,ubicacion,presencialidad,funciones,jornada,experiencia,tipo_contrato,salario,fecha_scrapeo,url,portal
0,Senior IT Ecosystem Developer (Blockchain Deve...,"[Blockchain, Full Stack Developer, Python, Eth...",Our client is a company creating an innovation...,madrid,Remoto parcial,This rol involves working in both Boston and M...,Jornada completa,5 años de experiencia,Indefinido,Salario no disponible,2024-01-27,https://talenthackers.net/spots/Digital/senior...,talenthacker
1,MDM Director,"[Master Data , Data Governance , Data Quality ...",Empresa multinacional de alimentos y productos...,barcelona,Remoto parcial,"Como Director de MDM (Master Data Management),...",Jornada completa,10 años de experiencia,Indefinido,Salario no disponible,2024-01-27,https://talenthackers.net/spots/Data/mdm-direc...,talenthacker
2,Data Engineer,"[Data Engineer, SQL, ETL]",Es el operador líder en el sector español de t...,Madrid,Remoto parcial,La principal función será garantizar que la in...,Jornada completa,2 años de experiencia,Indefinido,Salario no disponible,2024-01-27,https://talenthackers.net/spots/Data/data-engi...,talenthacker
3,Cybersecurity Incident Analyst,"[Ciberseguridad, SOC]",Empresa líder en el sector de las telecomunica...,Madrid,Remoto parcial,\nResponsable de analizar y gestionar los even...,Jornada completa,De 3 a 5 años de experiencia,Indefinido,Salario no disponible,2024-01-27,https://talenthackers.net/spots/SysOps/ciberse...,talenthacker
4,Ingeniero de Ciberseguridad,"[ciberseguridad, desarrollo]",La organización se dedica a la investigación y...,Derio,Remoto parcial,\nDesarrollo y optimización de software de cib...,Jornada completa,5 años de experiencia,Indefinido,Salario no disponible,2024-01-27,https://talenthackers.net/spots/Development/in...,talenthacker
5,Administrador de sistemas,"[AWS, Google Cloud Platform, Azure, Docker, Ku...",Es un centro de formación profesional que ofre...,lérida,Presencial,Buscan a un administrador de sistemas que se e...,Jornada completa,3 años de experiencia,Indefinido,Salario no disponible,2024-01-27,https://talenthackers.net/spots/Development/ad...,talenthacker
6,Desarrollador de Salesforce,"[Salesforce administrator, Salesforce Developm...",Es el operador líder en el sector español de t...,Madrid,Remoto parcial,\nDesarrollo e implementación de funcionalidad...,Jornada completa,3 años de experiencia,Indefinido,Salario no disponible,2024-01-27,https://talenthackers.net/spots/Development/de...,talenthacker
7,Software Architect Data & AI,"[Azure, Big Data]",Nuestro cliente es una empresa que se especial...,madrid,Remoto,"Responsable de diseño, desarrollo, despliegue ...",Jornada completa,3 años de experiencia,Indefinido,Salario no disponible,2024-01-27,https://talenthackers.net/spots/Data/software-...,talenthacker
8,Cloud Native Blueprint Leader,"[Arquitecto, Solución, AWS]",Empresa líder en el sector de las telecomunica...,Madrid,Remoto parcial,\nTu misión principal será:\nLiderar el proce...,Jornada completa,10 años de experiencia,Indefinido,Salario no disponible,2024-01-27,https://talenthackers.net/spots/SysOps/cloud-n...,talenthacker
9,Desarrollador ServiceNow ITSM/ITOM,"[ServiceNow, ITSM, ITOM]",Esta empresa española de servicios de consulto...,madrid,Remoto,"Diseñar, desarrollar y personalizar aplicacion...",Jornada completa,De 2 a 5 años de experiencia,Indefinido,Salario no disponible,2024-01-27,https://talenthackers.net/spots/Development/de...,talenthacker
